In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

3.7.9


In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from modules.datgan import DATWGANGP

import networkx as nx
import json
import beepy

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
dataset = 'Chicago'

In [5]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [6]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,3.93477,2,3,3,detached,6,0,30,1,4,PTE,20.166667
1,drive,2,SHOPPING,0.31557,3,3,3,detached,7,0,54,1,5,FTE,17.500000
2,drive,2,SHOPPING,0.28349,1,1,0,detached,3,0,80,1,3,PTE,9.333333
3,drive,2,OTHER,0.69417,2,2,0,detached,5,1,42,1,5,FTE,13.783333
4,passenger,1,SHOPPING,4.30666,2,2,1,detached,4,0,32,0,3,Unemployed,11.566667


In [7]:
if dataset is 'Chicago':
    continuous_columns = ["distance", "age", "departure_time"]
elif dataset is 'LPMC':
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [8]:
# personalised graph
graph = nx.DiGraph()

if dataset is 'Chicago':
    graph.add_edges_from([
        ("age", "license"),
        ("age", "education_level"),
        ("gender", "work_status"),
        ("education_level", "work_status"),
        ("education_level", "hh_income"),
        ("work_status", "hh_income"),
        ("hh_income", "hh_descr"),
        ("hh_income", "hh_size"),
        ("hh_size", "hh_vehicles"),
        ("hh_size", "hh_bikes"),
        ("work_status", "trip_purpose"),
        ("trip_purpose", "departure_time"),
        ("trip_purpose", "distance"),
        ("travel_dow", "choice"),
        ("distance", "choice"),
        ("departure_time", "choice"),
        ("hh_vehicles", "choice"),
        ("hh_bikes", "choice"),
        ("license", "choice"),
        # Non necessary links
        ("education_level", "hh_size"),
        ("work_status", "hh_descr"),
        ("work_status", "hh_size"),
        ("hh_income", "hh_bikes"),
        ("hh_income", "hh_vehicles"),
        ("trip_purpose", "choice")
    ])
elif dataset is 'LPMC':
    graph.add_edges_from([
        ("travel_year", "survey_year"),
        ("travel_date", "day_of_week"),
        ("day_of_week", "purpose"),
        ("purpose", "start_time_linear"),
        ("purpose", "cost_driving_con_charge"),
        ("purpose", "distance"),
        ("day_of_week", "driving_traffic_percent"),
        ("day_of_week", "cost_driving_con_charge"),
        ("start_time_linear", "driving_traffic_percent"),
        ("start_time_linear", "cost_driving_con_charge"),
        ("driving_traffic_percent", "cost_driving_con_charge"),
        ("female", "driving_license"),
        ("age", "bus_scale"),
        ("age", "car_ownership"),
        ("age", "driving_license"),
        ("age", "faretype"),
        ("driving_license", "car_ownership"),
        ("car_ownership", "fueltype"),
        ("fueltype", "cost_driving_con_charge"),
        ("fueltype", "cost_driving_fuel"),
        ("distance", "cost_driving_fuel"),
        ("distance", "dur_driving"),
        ("distance", "dur_walking"),
        ("distance", "dur_cycling"),
        ("distance", "dur_pt_access"),
        ("distance", "dur_pt_rail"),
        ("distance", "dur_pt_bus"),
        ("distance", "dur_pt_int"),
        ("dur_pt_bus", "cost_transit"),
        ("dur_pt_rail", "cost_transit"),
        ("pt_n_interchanges", "dur_pt_int"),
        ("pt_n_interchanges", "cost_transit"),
        ("faretype", "cost_transit"),
        ("bus_scale", "cost_transit"),
        ("car_ownership", "travel_mode"),
        ("age", "travel_mode"),
        ("cost_driving_con_charge", "travel_mode"),
        ("driving_traffic_percent", "travel_mode"),
        ("female", "travel_mode"),
        ("purpose", "travel_mode"),
        ("cost_transit", "travel_mode"),
        ("cost_driving_fuel", "travel_mode"),
        ("dur_driving", "travel_mode"),
        ("dur_walking", "travel_mode"),
        ("dur_cycling", "travel_mode"),
        ("dur_pt_access", "travel_mode"),
        ("dur_pt_rail", "travel_mode"),
        ("dur_pt_bus", "travel_mode"),
        ("dur_pt_int", "travel_mode")
    ])
    graph.add_node("travel_month")

In [9]:
output_folder = '../output/' + dataset + '/WGGP_WI/'

In [10]:
datgan = DATWGANGP(continuous_columns, max_epoch=1000, batch_size=500, output=output_folder, gpu=0)

In [11]:
test = datgan.fit(df, graph)

[1111 14:40:27 @DATSGAN.py:164] Preprocessing the data!
[1111 14:40:27 @data.py:370] Encoding continuous variable "age"...
[1111 14:40:27 @data.py:239]   Fitting model with 10 components
[1111 14:40:35 @data.py:266]   Predictions were done on 6 components => Fit with 6 components!
[1111 14:40:41 @data.py:271]   Predictions were done on 6 components => FINISHED!
[1111 14:40:41 @data.py:274]   Train VGM with full data
[1111 14:40:47 @data.py:383] Encoding categorical variable "gender"...
[1111 14:40:47 @data.py:383] Encoding categorical variable "travel_dow"...
[1111 14:40:47 @data.py:383] Encoding categorical variable "license"...
[1111 14:40:47 @data.py:383] Encoding categorical variable "education_level"...
[1111 14:40:47 @data.py:383] Encoding categorical variable "work_status"...
[1111 14:40:47 @data.py:383] Encoding categorical variable "hh_income"...
[1111 14:40:47 @data.py:383] Encoding categorical variable "trip_purpose"...
[1111 14:40:47 @data.py:383] Encoding categorical varia

[1111 14:41:28 @DATSGANModel.py:205] Creating cell for license (in-edges: 1; ancestors: 1)

[1111 14:41:29 @registry.py:126] gen/LSTM/license/FC input: [500, 100]
[1111 14:41:29 @registry.py:134] gen/LSTM/license/FC output: [500, 50]
[1111 14:41:29 @registry.py:126] gen/LSTM/license/FC_prob input: [500, 50]
[1111 14:41:29 @registry.py:134] gen/LSTM/license/FC_prob output: [500, 2]
[1111 14:41:29 @registry.py:126] gen/LSTM/license/FC_input input: [500, 2]
[1111 14:41:29 @registry.py:134] gen/LSTM/license/FC_input output: [500, 100]
[1111 14:41:29 @DATSGANModel.py:205] Creating cell for education_level (in-edges: 1; ancestors: 1)
[1111 14:41:29 @registry.py:126] gen/LSTM/education_level/FC input: [500, 100]
[1111 14:41:29 @registry.py:134] gen/LSTM/education_level/FC output: [500, 50]
[1111 14:41:29 @registry.py:126] gen/LSTM/education_level/FC_prob input: [500, 50]
[1111 14:41:29 @registry.py:134] gen/LSTM/education_level/FC_prob output: [500, 6]
[1111 14:41:29 @registry.py:126] gen/LST

[1111 14:41:29 @registry.py:126] gen/LSTM/distance/FC_prob input: [500, 50]
[1111 14:41:29 @registry.py:134] gen/LSTM/distance/FC_prob output: [500, 6]
[1111 14:41:29 @registry.py:126] gen/LSTM/distance/FC_input input: [500, 12]
[1111 14:41:29 @registry.py:134] gen/LSTM/distance/FC_input output: [500, 100]
[1111 14:41:29 @DATSGANModel.py:205] Creating cell for hh_vehicles (in-edges: 2; ancestors: 6)
[1111 14:41:29 @registry.py:126] gen/LSTM/concat-hh_vehicles/FC_inputs input: [500, 200]
[1111 14:41:29 @registry.py:134] gen/LSTM/concat-hh_vehicles/FC_inputs output: [500, 100]
[1111 14:41:29 @registry.py:126] gen/LSTM/concat-hh_vehicles/FC_states input: [500, 200]
[1111 14:41:29 @registry.py:134] gen/LSTM/concat-hh_vehicles/FC_states output: [500, 100]
[1111 14:41:29 @registry.py:126] gen/LSTM/concat-hh_vehicles/FC_h_outputs input: [500, 200]
[1111 14:41:29 @registry.py:134] gen/LSTM/concat-hh_vehicles/FC_h_outputs output: [500, 100]
[1111 14:41:29 @registry.py:126] gen/LSTM/hh_vehicles/

[1111 14:41:33 @base.py:209] Setup callbacks graph ...






[1111 14:41:33 @summary.py:46] [MovingAverageSummary] 5 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[1111 14:41:33 @summary.py:93] Summarizing collection 'summaries' of size 6.


[1111 14:41:34 @base.py:230] Creating the session ...


Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.



[1111 14:41:36 @base.py:236] Initializing the session ...
[1111 14:41:36 @base.py:243] Graph Finalized.


[1111 14:41:36 @concurrency.py:38] Starting EnqueueThread QueueInput/input_queue ...

[1111 14:41:36 @base.py:275] Start Epoch 1 ...


 35%|##############################3                                                       |6/17[00:02<00:08, 1.23it/s]

...

[1111 14:54:11 @base.py:285] Epoch 999 (global_step 20379) finished, time:0.569 second.
[1111 14:54:11 @saver.py:79] Model saved to ../output/Chicago/WGGP_WI2/model\model-20379.


[1111 14:54:12 @monitor.py:467] GAN_loss/d_loss: -1.0594
[1111 14:54:12 @monitor.py:467] GAN_loss/g_loss: 5.5133
[1111 14:54:12 @monitor.py:467] GAN_loss/gradient_penalty: 0.036098
[1111 14:54:12 @monitor.py:467] GAN_loss/gradient_rms: 1.035
[1111 14:54:12 @monitor.py:467] GAN_loss/kl_div: 0.052595
[1111 14:54:12 @monitor.py:467] QueueInput/queue_size: 50
[1111 14:54:12 @base.py:275] Start Epoch 1000 ...


100%|#####################################################################################|17/17[00:00<00:00,29.92it/s]

[1111 14:54:12 @base.py:285] Epoch 1000 (global_step 20399) finished, time:0.569 second.
[1111 14:54:12 @saver.py:79] Model saved to ../output/Chicago/WGGP_WI2/model\model-20399.


[1111 14:54:12 @monitor.py:467] GAN_loss/d_loss: -1.0396
[1111 14:54:12 @monitor.py:467] GAN_loss/g_loss: 5.5125
[1111 14:54:12 @monitor.py:467] GAN_loss/gradient_penalty: 0.036826
[1111 14:54:12 @monitor.py:467] GAN_loss/gradient_rms: 1.0387
[1111 14:54:12 @monitor.py:467] GAN_loss/kl_div: 0.050601
[1111 14:54:12 @monitor.py:467] QueueInput/queue_size: 50
[1111 14:54:12 @base.py:289] Training has finished!

[1111 14:54:12 @DATSGANModel.py:205] Creating cell for age (in-edges: 0; ancestors: 0)
[1111 14:54:12 @DATSGANModel.py:205] Creating cell for gender (in-edges: 0; ancestors: 0)
[1111 14:54:13 @DATSGANModel.py:205] Creating cell for travel_dow (in-edges: 0; ancestors: 0)
[1111 14:54:13 @DATSGANModel.py:205] Creating cell for license (in-edges: 1; ancestors: 1)
[1111 14:54:13 @DATSGANModel.py:205] Creating cell for education_level (in-edges: 1; ancestors: 1)
[1111 14:54:13 @DATSGANModel.py:205] Creating cell for work_status (in-edges: 2; ancestors: 3)
[1111 14:54:13 @DATSGANModel.py:

In [12]:
datgan.save('trained', force=True)

[1111 14:54:30 @DATSGAN.py:362] Model saved successfully.


In [13]:
beepy.beep(6)